In [108]:
# 코드 작성일 : 24.7.25
pd.set_option('display.max_rows', None) # 전체 컬럼 모두 출력
import os

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
    original_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(original_dir))))
    os.chdir(original_dir)
exec(open('setup/default.py').read())

In [109]:
menu_list = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/spaced_menu.csv')

# Replacement rules
replacements = {
    "돈가스": "돈까스",
    "자장": "짜장",
    "코든블루": "코돈브루",
    "코든브루": "코돈브루",
    "코돈블루": "코돈브루",
    "플래이트": "",
    "플레이트": "",
    "플래터": "",
    "플레터": "",
    "정식": "",
    "소시지": "소세지",
    "돈유": "돈육",
    "함바그" : "함박",
    "커틀렛" : "커틀릿",
    "오무" : "오므"
}

def replace_text(text, replacements):
    for key, value in replacements.items():
        text = text.replace(key, value)
    # Remove digits
    text = ''.join([i for i in text if not i.isdigit()])
    return text

menu_list['first_menu'] = menu_list['first_menu'].apply(lambda x: replace_text(x, replacements))
menu_list['first_menu_sp'] = menu_list['first_menu_sp'].apply(lambda x: replace_text(x, replacements))


In [110]:
high_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/high_cat.csv').drop_duplicates()['high_cat'].tolist()
med_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/med_cat.csv').drop_duplicates()['med_cat'].tolist()
ingredient = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/ingredient.csv').drop_duplicates()['ingredient'].tolist()

In [111]:
# Split 'first_menu_sp' into lists
menu_list ['by_value'] = menu_list ['first_menu_sp'].apply(lambda x: x.split())
menu_list ['etc'] = menu_list ['first_menu_sp'].apply(lambda x: x.split())

In [112]:
# Define a function to find and remove the matching high_cat value
def find_and_remove_cat(etc_list, cat_values):
    for item in reversed(etc_list):
        if item in cat_values:
            etc_list.remove(item)  # Remove the item from the original list
            return item
    return None

def find_and_remove_all_cat(etc_list, cat_values):
    matched_items = [item for item in etc_list if item in cat_values]
    etc_list[:] = [item for item in etc_list if item not in cat_values]
    return matched_items if matched_items else None

# Apply the function to the etc column
menu_list['high_cat'] = menu_list['etc'].apply(lambda x: find_and_remove_cat(x, high_cat) if x else None)
menu_list['med_cat'] = menu_list['etc'].apply(lambda x: find_and_remove_cat(x, med_cat) if x else None)
menu_list['ingredient'] = menu_list['etc'].apply(lambda x: find_and_remove_all_cat(x, ingredient) if x else None)


In [113]:
# Flatten the list of lists in the 'etc' column
all_elements = [element for sublist in menu_list['etc'] for element in sublist]
from collections import Counter
# Calculate frequency of each element
element_counts = Counter(all_elements)

# Convert to DataFrame and sort by count
element_counts_df = pd.DataFrame(element_counts.items(), columns=['Element', 'Count'])
element_counts_df = element_counts_df.sort_values(by='Count', ascending=False).reset_index(drop=True)
element_counts_df.to_csv('/Users/al01989093/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/entity/test2.csv')

In [117]:
menu_list

,first_menu,first_menu_sp,by_value,etc,high_cat,med_cat,ingredient,last_value
0,뼈없는감자탕,뼈없는 감자 탕,"[뼈없는, 감자, 탕]",[뼈없는],탕,None,[감자],탕
1,닭곰탕,닭 곰 탕,"[닭, 곰, 탕]",[],탕,곰,[닭],탕
2,콩비지찌개,콩비지 찌개,"[콩비지, 찌개]",[],찌개,None,[콩비지],찌개
3,마파두부덮밥,마파두부 덮 밥,"[마파두부, 덮, 밥]",[],밥,덮,[마파두부],밥
4,돼지국밥,돼지 국 밥,"[돼지, 국, 밥]",[],밥,국,[돼지],밥
5,옛날카레라이스,옛날 카레 라이스,"[옛날, 카레, 라이스]",[옛날],라이스,카레,None,라이스
6,통마늘돼지불백,통 마늘 돼지 불백,"[통, 마늘, 돼지, 불백]",[통],불백,None,"[마늘, 돼지]",불백
7,돼지양념구이,돼지 양념 구이,"[돼지, 양념, 구이]",[양념],구이,None,[돼지],구이
8,가츠동돈부리,가츠 동 돈부리,"[가츠, 동, 돈부리]",[동],돈부리,None,[가츠],돈부리
9,장조림버터비빔밥,장조림 버터 비빔 밥,"[장조림, 버터, 비빔, 밥]",[],밥,비빔,"[장조림, 버터]",밥


In [115]:
# Split 'first_menu_sp' into lists
menu_list ['by_value'] = menu_list ['first_menu_sp'].apply(lambda x: x.split())

# Extract the last element from each list into a new column 'last_value'
menu_list['last_value'] = menu_list ['by_value'].apply(lambda x: x[-1])


In [116]:

menu_list.groupby(['last_value']).size().to_frame('size').reset_index().sort_values(by='size',ascending=False).\
to_csv(original_dir + '/data/feature_lab/menu_map/entity/test.csv',index=False)